In [ ]:
%matplotlib inline
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy

In [ ]:
from simlightcurve.curves import GaussExp as Gred
from simlightcurve.solvers import find_rise_t, find_peak
from scipy.stats import norm, multivariate_normal
import scipy.optimize as op
import emcee

In [ ]:
seaborn.set_context('talk')
current_palette = seaborn.color_palette()

In [ ]:
e1=1
e2=1
xcov=0.5
cov = np.asarray([[e1,xcov],
                  [xcov,e2]])
print "Det:", e1*e2-xcov*xcov
true_mean = np.random.random(size=2)
print "True Mean: ", true_mean
rv = multivariate_normal(mean=true_mean, cov=cov)
x = np.linspace(-3,3,100)
y = np.linspace(-4,4,100)
grid = np.dstack(np.meshgrid(x,y))
# print x[0],x[1]
# y = np.sqrt(x[0]**2+x[1]**2)

seaborn.set_palette('husl')
cmap = plt.cm.get_cmap('afmhot')
plt.contourf(x,y,rv.pdf(grid),25,cmap=cmap)
plt.ylabel('i')
plt.xlabel('j')
plt.axvline(mean[0], ls='--')
plt.axhline(mean[1], ls='--')
plt.colorbar()

n_samples = 1
data = np.atleast_2d(rv.rvs(n_samples))
# print "data:", data
print "datashape", data.shape
plt.scatter(data[:,0],data[:,0])
# plt.scatter(data[0,:],data[1,:])



In [ ]:
icov = np.linalg.inv(cov)
icov

In [ ]:
u,s,v = np.linalg.svd(cov)
print u
print s
print v

In [ ]:
u-v

In [ ]:
np.dot(cov,icov)

In [ ]:
np.linalg.det(cov)

In [ ]:
# np.linalg.svd(cov)

In [ ]:
def lnprob(mu,x,icov):
    diff = x-mu
    return -np.dot(diff,np.dot(icov,diff))/2.0
    

In [ ]:
icov

In [ ]:
# data = np.array([ 0.80008131,  0.942178  ])
diff = data - true_mean

test = np.asarray(
    [
                    [1,.1],
                    [0.2,2],
#                     [1,1],
                   ]
                  )
test = np.atleast_2d(test).reshape(2,-1)
print "Testshape", test.shape
print "Test:", test

np.dot(icov,test)
# np.(data - true_mean)

In [ ]:
np.dot(test.T,np.dot(icov,test))

In [ ]:
test*np.dot(icov,test)

In [ ]:
print data.shape

In [ ]:
lnprob(true_mean,data,icov)

In [ ]:
ndim = 2  # number of parameters in the model
nwalkers = 50  # number of MCMC walkers
nburn = 100  # "burn-in" period to let chains stabilize
nsteps = 500  # number of MCMC steps to take

mean_init = [data[0] + 1e-5*np.random.randn(ndim) for i in range(nwalkers)]
# pos = [0.5 + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
# set theta near the maximum likelihood, with 
# sampler.reset()
# sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likelihood, args=(data,))
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(data,icov))
sampler.reset()
_=sampler.run_mcmc(mean_init, nsteps)
for walker in sampler.chain:
    plt.plot(walker)